In [19]:
import numpy as np
import pandas as pd

In [20]:
TARGET_COL = 'scalar_coupling_constant'
N_FOLDS = 8
VERSION = 2
FOLD_IDS = [1, 2, 3, 4, 5, 6, 7, 8]

In [21]:
PATH = '../tmp/'
SUB_PATH = '../submissions/'
OOF_PATH = '../oofs/'

In [22]:
train_df = pd.read_csv(PATH+'train_proc_df.csv', index_col=0)
val_idxs = pd.read_csv(PATH+'val_idxs_8_fold_cv.csv', index_col=0)

/anaconda2/envs/python36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [23]:
sub_files = [f'mpnn-v{VERSION}-idx{i}-submission' for i in FOLD_IDS]
oof_files = [f'mpnn-v{VERSION}-idx{i}-oof' for i in FOLD_IDS]
preds = pd.concat([pd.read_csv(SUB_PATH+f+'.csv', index_col=0) for f in sub_files], axis=1)
oofs = pd.concat([pd.read_csv(OOF_PATH+f+'.csv', index_col=0) for f in oof_files], axis=1)

/anaconda2/envs/python36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [24]:
def group_mean_log_mae(y_true, y_pred, types, return_groups=False):
    maes = (y_true - y_pred).abs().groupby(types).mean()
    if return_groups:
        return np.log(maes).mean(), np.log(maes)
    else:
        return np.log(maes).mean()

In [25]:
oof = pd.DataFrame(train_df['molecule_id'])
oof[TARGET_COL] = np.nan
for i, idx in zip(range(N_FOLDS), FOLD_IDS):
    idx = oof['molecule_id'].isin(val_idxs.iloc[:,idx-1].dropna().astype(int))
    oof[TARGET_COL][idx] = oofs.iloc[:,i].dropna().values

gml_mae, gml_maes = group_mean_log_mae(train_df[TARGET_COL], oof[TARGET_COL], train_df['type'], True)
gml_maes.index = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN']
gml_mae, gml_maes

/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


(-2.640727070215746, 1JHC   -1.767384
 2JHH   -3.145183
 1JHN   -2.103106
 2JHN   -2.872853
 2JHC   -2.619462
 3JHH   -3.016509
 3JHC   -2.535968
 3JHN   -3.065352
 Name: scalar_coupling_constant, dtype: float64)

In [26]:
sub_ens = pd.DataFrame(preds.mean(axis=1), columns=[TARGET_COL])
sub_ens.head()

,scalar_coupling_constant
id,
4658147,15.146106
4658148,189.054861
4658149,1.804049
4658150,189.054860
4658151,15.146106


In [27]:
sub_ens.to_csv(f'{SUB_PATH}mpnn-v{VERSION}-submission.csv')

In [32]:
oof_swa = pd.read_csv(OOF_PATH+f'mpnn-swa-v{VERSION}-idx{5}-oof'+'.csv', index_col=0)
idx = oof['molecule_id'].isin(val_idxs.iloc[:,4].dropna().astype(int))
print(group_mean_log_mae(train_df[TARGET_COL][idx], oof_swa.iloc[:,0].values, train_df['type'][idx], True))
print(group_mean_log_mae(train_df[TARGET_COL][idx], oofs.iloc[:,4].dropna().values, train_df['type'][idx], True))

(-2.6567292615048768, type
0   -1.778491
1   -3.154441
2   -2.114161
3   -2.935002
4   -2.635238
5   -3.027673
6   -2.553026
7   -3.055802
Name: scalar_coupling_constant, dtype: float64)
(-2.6544766968056157, type
0   -1.776686
1   -3.150140
2   -2.115865
3   -2.931155
4   -2.632283
5   -3.023888
6   -2.549252
7   -3.056545
Name: scalar_coupling_constant, dtype: float64)
